In [1]:
import re
import time
from typing import Iterator
import requests
import lxml
import lxml.html
import cssselect
from pymongo import MongoClient

import pandas as pd

In [32]:
#クローラー

#各年directoryURLのリスト
def main():
    
    client = MongoClient('localhost', 27017)  # ローカルホストのMongoDBに接続する。
    collection = client.scraping.coe_results_2021  # scrapingデータベースのcoeコレクションを得る。
    collection.create_index('key',unique=True) # データを一意に識別するキーを格納するkeyフィールドにユニークなインデックスを作成する。
    
    headers = requests.utils.default_headers()
    headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Safari/605.1.15'
    session = requests.Session()
    
    #htmlファイル取得
    response = session.get("https://cupofexcellence.org/competition-auction-results/")
    html = lxml.html.fromstring(response.text)
    
    #全てのcountry-2021urlリスト
    urls_country_2021=[]
    for a_2021 in html.cssselect('li.menu-item.menu-item-type-custom.menu-item-object-custom.current-menu-item.menu-item-has-children a:contains("2021")'):
        urls_country_2021.append(a_2021.get('href'))
    urls_country_2021=set(urls_country_2021)
    
    #2021年-各国ごと処理
    for url_country_2021 in urls_country_2021:
        print(url_country_2021)
        time.sleep(7)
        response_country_2021 = session.get(url_country_2021)
        html_country_2021 = lxml.html.fromstring(response_country_2021.text)
        
        #あるcountry-2021の各rank-urlリスト
        urls_rank_country_2021=[]
        for a_rank_country_2021 in html_country_2021.cssselect('div.vc_tta-panel-body a'):
            urls_rank_country_2021.append(a_rank_country_2021.get('href'))
        
        #scrape-detail-page
        for url_rank_country_2021 in urls_rank_country_2021:
            if not 'cupofexcellence' in str(url_rank_country_2021):
                continue
            print(url_rank_country_2021)
            key = extract_key(url_rank_country_2021)  # URLからキーを取得する。（rank_country_2021）
            coe_results = collection.find_one({'key': key})  # MongoDBからkeyに該当するデータを探す。
           
            if not coe_results:  # MongoDBに存在しない場合、詳細ページをクロールする。
                time.sleep(7)
                response_rank_country_2021 = session.get(url_rank_country_2021)  # 詳細ページを取得する。
                result = scrape_detail_page(response_rank_country_2021, key)
                result = {k.replace('.',''): v for k, v in result.items() if type(k) != int}
                collection.insert_one(result)  # 表の情報をMongoDBのcoeコレクションに保存する。
                    
def scrape_detail_page(response_rank_country_2021, key) -> dict:
    html=lxml.html.fromstring(response_rank_country_2021.text)
    data={}
    #Main Information
    for li in html.cssselect('div.element.table-block li'):
        l=list(map(lambda x:x.strip(), li.text_content().splitlines()))
        l.remove('')
        data.update({l[0]: l[1]})
    #Location
    location=html.cssselect('div.map-block-address p')
    if len(location)!=0:
        data.update({'Location': location[0].text_content()})
    #Auction
    auction=list(map(lambda p:p.text_content(), html.cssselect('div.element.content-block.plain-text-content p')))
    try:
        auction.remove('')
    except ValueError as error:
        pass
    if len(auction)!=0:
        data.update({'Auction': auction[0]})
    #key, url
    data.update({'key': key})
    return data
    

def extract_key(url: str) -> str:
    """
    URLからキー（URLの末尾のISBN）を抜き出す。
    """
    m = re.search(r'/([^/]+)/$', url)  # 最後の/から文字列末尾までを正規表現で取得。
    return m.group(1)

if __name__ == '__main__':
    main()

https://cupofexcellence.org/colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/obraje-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/obraje-colombia-2021-2/
https://farmdirectory.cupofexcellence.org/listing/la-siria-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/la-siria-colombia-2021-2/
https://farmdirectory.cupofexcellence.org/listing/el-zafiro-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/guayacanes-el-tejar-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/el-casino-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/la-guaca-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/la-piragua-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/la-miel-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/el-cedro-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/la-fortuna-colombia-2021/
https://farmdirectory.cupofexcellence.org/listing/san